In [48]:
import numpy as np
import skfuzzy as fuzz
import pandas as pd
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler





In [49]:
#Acesso ao dataset
music = pd.read_csv('dataset.csv', index_col=0) #chama o ficheiro e remove coluna desnecessária (index_col)

#Garantir que o acesso foi bem sucedido (faz print das primeiras 5 linhas do dataset)
print(music.head())
music.shape

                 track_id                 artists  \
0  5SuOikwiRyPMVoIQDJUgSV             Gen Hoshino   
1  4qPNDBW1i3p13qLCt0Ki3A            Ben Woodward   
2  1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson;ZAYN   
3  6lfxq3CG4xtTiEg7opyCyx            Kina Grannis   
4  5vjLSffimiIP26QG5WcN2K        Chord Overstreet   

                                          album_name  \
0                                             Comedy   
1                                   Ghost (Acoustic)   
2                                     To Begin Again   
3  Crazy Rich Asians (Original Motion Picture Sou...   
4                                            Hold On   

                   track_name  popularity  duration_ms  explicit  \
0                      Comedy          73       230666     False   
1            Ghost - Acoustic          55       149610     False   
2              To Begin Again          57       210826     False   
3  Can't Help Falling In Love          71       201933     False   
4   

(114000, 20)

In [ ]:
##limpeza do dataset
# Contar quantas linhas duplicadas existem (mesmo nome + mesmo artista)
duplicadas_antes = music.duplicated(subset=["track_name", "artists"]).sum()
print(f"Antes da limpeza: {duplicadas_antes} músicas duplicadas")

# Remover duplicados (mesmo nome e mesmo artista)
music = music.drop_duplicates(subset=["track_name", "artists"], keep="first")

print(music.shape)

#remover tracks que não são musica (podcasts, audio books)
music = music[music['speechiness'] <= 0.66]

print(f'Depois de filtrar para incluir apenas as músicas: {music.shape}')


#valores em falta por coluna
print(music.isnull().sum()) 
music = music.dropna()  #por serem poucos


Antes da limpeza: 32656 músicas duplicadas
(81344, 20)
depois de filtrar só as musicas: (80483, 20)
track_id            0
artists             1
album_name          1
track_name          1
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64


In [51]:
#defenir as features
f0 = music['danceability'].values   #[0,1]
f1 = music['energy'].values         #[0,1]
f2 = music['key'].values            #[-1,11]
f3 = music['loudness'].values       #dB
f4 = music['mode'].values           #binary
f5 = music['acousticness'].values   #[0,1]
f6 = music['tempo'].values          #bpm
f7 = music['time_signature'].values #[0,5]
f8 = music['track_genre'].values    #classes

y = music.values[:, 16]    #valence-target


# matriz features
x = music.values[:, [7, 8, 9, 10, 11, 13, 17, 18, 19]]
#x = music[['danceability', 'energy', 'key',.....]]    
'-------------------------------------melhor metodo?'

print(f1)
print(f8)
print(y)
print(x)
music.shape

[0.461 0.166 0.359 ... 0.329 0.506 0.487]
['acoustic' 'acoustic' 'acoustic' ... 'world-music' 'world-music'
 'world-music']
[0.715 0.267 0.12 ... 0.743 0.413 0.708]
[[0.676 0.461 1 ... 87.917 4 'acoustic']
 [0.42 0.166 1 ... 77.489 4 'acoustic']
 [0.438 0.359 0 ... 76.332 4 'acoustic']
 ...
 [0.629 0.329 0 ... 132.378 4 'world-music']
 [0.587 0.506 7 ... 135.96 4 'world-music']
 [0.526 0.487 1 ... 79.198 4 'world-music']]


(80482, 20)

In [52]:
#verificaçaõ da limpeza
duplicadas_depois = music.duplicated(subset=["track_name", "artists"]).sum()
print(f"Depois da limpeza: {duplicadas_depois} músicas duplicadas")


Depois da limpeza: 0 músicas duplicadas


In [53]:
#train, validation e test split
test_size = 0.3
val_size = 0.5   #será metade dos dados de teste
#Seprarar o conjunto de treino (70%) e o temporário (30%)
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size = test_size, random_state = 42)
#Separar o conjunto temporário em teste (15%) e validação (15%)
x_val, x_test, y_val, y_test = train_test_split(x, y, test_size = val_size, random_state = 42)

'test será o conjunto nunca visto pelo algoritmo até ao fim'

'test será o conjunto nunca visto pelo algoritmo até ao fim'

In [54]:
#Normalizar varivaeis Key, loudness, tempo, time_signature
scaler = MinMaxScaler()
x_train[ :, [ 2, 3, 6, 7]] = scaler.fit_transform(x_train[ :, [ 2, 3, 6, 7]])
#x_val[ :, [ 2, 3, 6, 7]] = scaler.transform(x_val[ :, 2, 3, 6, 7])

print(x_train)
'''----------PORQUE NÃO POSSO FAZER A NORMALIZAÇÃO ANTES DE SEPARAR ENTRE TESTE E TREINO
OU SEJA DÁ PARA O FAZER SEM LEVAR A OVER FITTING?'''

[[0.497 0.294 0.4545454545454546 ... 0.7149261213286655 0.8 'cantopop']
 [0.586 0.695 0.09090909090909091 ... 0.6488831911641437 0.8 'ska']
 [0.667 0.503 0.0 ... 0.5917484344953404 0.8 'edm']
 ...
 [0.788 0.745 0.18181818181818182 ... 0.5012943148759923 0.8 'techno']
 [0.564 0.731 0.8181818181818182 ... 0.3904968525549365 0.8 'acoustic']
 [0.771 0.739 0.8181818181818182 ... 0.5342603093207107 0.8 'comedy']]


'----------PORQUE NÃO POSSO FAZER A NORMALIZAÇÃO ANTES DE SEPARAR ENTRE TESTE E TREINO\nOU SEJA DÁ PARA O FAZER SEM LEVAR A OVER FITTING?'

In [55]:
#quais os track_genres

estilos = []

for nome in music['track_genre']:
  if nome in estilos:
    None
  else:
    estilos.append(nome)


print(estilos)

#TEMOS QUE FILTRAR ....clustering

['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'idm', 'indian', 'indie-pop', 'indie', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metalcore', 'minimal-techno', 'mpb', 'new-age', 'opera', 'pagode', 'party', 'piano', 'pop-film', 'pop', 'power-pop', 'progressive-house', 'psych-rock', 'punk-rock', 'punk', 'r-n-b', 'reggae', 'reggaeton', 'rock-n